# Bedrock Prompt Examples for Media & Entertainment

In this notebook, we include different example use cases for Media & Entertainment using Amazon Bedrock.

These sample use cases involve different tasks and prompt engeering techniques, as follows:
1. **Generate recommendations based on metadata**
    - **Task:** Text Generation
    - **Prompt Technique:** Zero-shot
2. **Estimate audience for TV shows based on historical data**
    - **Task:** Complex Reasoning
    - **Prompt Technique:** Chain-of-Thoughts (CoT)
3. **Create a question answering assistant for an entertainment company**
    - **Task:** Question Answering with Dialogue Asistant (without memory)
    - **Prompt Technique:** Few-shot
4. **Summarize and classify content from media files transcription**
    - **Task:** Text Summarization & Text Classification
    - **Prompt Technique:** Zero-shot
5. **Create splash pages describing upcoming events**
    - **Task:** Code Generation
    - **Prompt Technique:** Zero-shot

This notebook uses Amazon Bedrock, make sure you have access to the service and have the dependencies installed if required. You can check details for this in the [Amazon Bedrock Workshop](https://github.com/aws-samples/amazon-bedrock-workshop/blob/main/00_Intro/bedrock_boto3_setup.ipynb).

First, let's setup the Bedrock client.

In [1]:
import sys, os

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock_utils

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

os.environ["AWS_DEFAULT_REGION"] = "us-east-1"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
os.environ["BEDROCK_ASSUME_ROLE"] = "arn:aws:iam::114798159153:role/BedrockRemote"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."

bedrock = bedrock_utils.get_bedrock(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

Create new client
  Using region: us-east-1
  Using role: arn:aws:iam::114798159153:role/BedrockRemote ... successful!
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-east-1.amazonaws.com)


In [2]:
bedrock.list_foundation_models()

{'ResponseMetadata': {'RequestId': '873423df-06d2-41e8-ae15-2432b1fa2848',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 11 Aug 2023 14:43:17 GMT',
   'content-type': 'application/json',
   'content-length': '1166',
   'connection': 'keep-alive',
   'x-amzn-requestid': '873423df-06d2-41e8-ae15-2432b1fa2848'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-e1t-medium',
   'modelId': 'amazon.titan-e1t-medium'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/stability.stable-diffusion-xl',
   'modelId': 'stability.stable-diffusion-xl'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/ai21.j2-grande-instruct',
   'modelId': 'ai21.j2-grande-instruct'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/ai21.j2-jumbo-instruct',
   'modelId': 'ai21.j2-jumbo-i

**Note:** We'll use a utility function 'call_bedrock' for invoking Bedrock with the proper parameters.

It accepts the following arguments:
* Bedrock client (leave as 'bedrock' in this notebook)
* Prompt
* ModelId (if not specified, defaults to Claude v2)
* Temperature (if not specified, defaults to 0)
* Top P (if not specified, defaults to 0.9)

You can adjust any other parameters by modifying the function [here](./utils/bedrock_utils.py)

## 1. Generate recommendations based on metadata

**Use Case:** A media & entertainment company wants to generate recommendations of TV shows for their audience based on some metadata of viewers, e.g. country, age-range, and theme.

**Task:** Text Generation

**Prompt Technique:** Zero-shot

In [3]:
prompt_data ="""
Human:
Generate a list of 10 recommended TV shows to watch considering the information in the <metadata></metadata> XML tags, and include a very brief description of each recommendation.

<metadata>
Country is UK
Age range between 20-30
Shows must be about sports
</metadata>

Assistant:
"""

In [4]:
response, latency = bedrock_utils.call_bedrock(bedrock, prompt_data)
print(response, "\n\n", "Inference time:", latency)

 Here are 10 recommended TV shows to watch based on the provided metadata:

1. Match of the Day (BBC) - Longest running football highlights show in the UK covering the Premier League.

2. A League of Their Own (Sky) - Comedy sports quiz show with celebrity guests and sketches. 

3. Formula 1 (Sky) - Coverage of every Formula 1 Grand Prix race.

4. NFL (Sky/BBC) - Live coverage of American football games from the NFL. 

5. Cricket (Sky) - Live coverage of cricket matches and tournaments.

6. Rugby Union (BBC/ITV) - Coverage of rugby union matches, Six Nations tournament. 

7. Darts (Sky) - Live coverage of professional darts tournaments.

8. Golf (Sky) - Live golf tournaments coverage including the majors.

9. NBA (Sky/BBC) - Basketball games from the NBA shown live or highlights.

10. Rugby League (Sky/BBC) - Rugby league matches from Super League and internationals. 

 Inference time: 0:00:09.324579


------

## 2. Estimate audience for TV shows based on historical data

**Use Case:** A media & entertainment company wants to estimate the audience levels they could have for the next day based on historical information and shows metadata.

**Task:** Complex Reasoning

**Prompt Technique:** Chain-of-Thoughts (CoT)

In [12]:
prompt_data ="""
Human: Last week, the viewers of 3 television networks were with the following data:
- Monday: SportsTV 6500, NewsTV 3200, EntertainmentTV 4150
- Tuesday: SportsTV 6400, NewsTV 3300, EntertainmentTV 4100
- Wednesday: SportsTV 6300, NewsTV 3400, EntertainmentTV 4250

Question: How many viewers can we expect next Friday on SportsTV?
Answer: According to the numbers given and without having more information, there is a daily decrease of 100 viewers for SportsTV.
If we assume that this trend will continue for the next few days, we can expect 6200 viewers for the next day which is Thursday, and
therefore 6100 viewers for Friday.

Question: How many viewers can we expect on Saturday in each of the channels? think step-by-step, and provide recommendations for increasing the viewers
Assistant:
Answer:
"""

In [13]:
response, latency = bedrock_utils.call_bedrock(bedrock, prompt_data)
print(response, "\n\n", "Inference time:", latency)

Step 1) Identify the trend for each channel:
- SportsTV: Decreasing by 100 viewers each day
- NewsTV: Increasing by 100 viewers each day  
- EntertainmentTV: Increasing by 50 viewers each day

Step 2) Extrapolate the trends to Saturday:
- SportsTV: 6100 on Friday - 100 = 6000 viewers on Saturday
- NewsTV: 3400 on Wednesday + 200 (2 days of increase) = 3600 viewers on Saturday  
- EntertainmentTV: 4250 on Wednesday + 100 (2 days of increase) = 4350 viewers on Saturday

Step 3) Recommendations to increase viewers:

For SportsTV:
- Add new and exciting sporting events or personalities to attract more viewers
- Promote upcoming matches/games on social media to generate buzz 

For NewsTV: 
- Increase coverage of breaking news and live events to boost real-time viewership
- Analyze data to identify topics of high interest and produce more content around those

For EntertainmentTV:
- Partner with popular celebrities or shows to attract their fanbase
- Optimize programming schedule to retain v

------

## 3. Create a question answering assistant for an entertainment company

**Use Case:** An entertainment company wants to create a bot capable of answering questions about the shows available, based on the information for these shows.

**Task:** Question Answering with Dialogue Asistant (no memory)

**Prompt Technique:** Few-shot

In [14]:
prompt_data ="""
Context: The shows available are as follows
1. Circus, showing at the Plaza venue, assigned seating, live at 8pm on weekends
2. Concert, showing at the Main Teather, assigned seating, live at 10pm everyday
3. Basketball tricks, showing at the Sports venue, standing seating, live at 5pm on weekdays

Instruction: Answer any questions about the shows available. If you don't know the answer just say 'Apologies, but I don't have the answer for that. Please contact our team by phone.'

Assistant: Welcome to Entertainment Tonight, how can I help you?
Human: Hi, I would like to know what are the shows available please.
Assistant: Of course, right now we have the Circus, the Concert, and the Basketball tricks shows.
Human: Thank you. I would like to know when and where are those available please.
Assistant:
"""

In [15]:
response, latency = bedrock_utils.call_bedrock(bedrock, prompt_data)
print(response, "\n\n", "Inference time:", latency)

 Here are the details on each of the shows we currently have:

Circus - This show is at the Plaza venue with assigned seating. It runs live at 8pm on weekends.

Concert - This show is at the Main Theater, also with assigned seating. It runs live every night at 10pm. 

Basketball Tricks - This show is at the Sports venue with standing room only seating. It runs live on weekdays at 5pm.

Let me know if you need any additional details on the shows or venues! I'd be happy to provide more information. 

 Inference time: 0:00:03.472495


------

## 4. Generate content summary based on transcriptions from media files

**Use Case:** A media company needs to generate summaries of the audio transcription for audio and video files, to be sent to their content team. They also want to classify this content according to specific categories.

**Task:** Text Summarization & Text Classification

**Prompt Technique:** Zero-shot

#### (Pre-requisite)

First, we will need to transcribe the media files. You can e.g. use Amazon Transcribe for this task following examples like this: https://docs.aws.amazon.com/code-library/latest/ug/transcribe_example_transcribe_StartTranscriptionJob_section.html

For our sample we will start from an interview transcription in the file "interview.txt".

In [16]:
f = open("interview.txt", "r").read()
print(f)

Presenter: (00:00)
Thank you for doing This Christmas Will Be Different. I appreciate you doing that. We hung out all day together.

Actor Smith: (00:04)
How fun was that?

Presenter: (00:05)
It was the greatest. We always…

Actor Smith: (00:07)
Two take maximum. Two take maximum.

Presenter: (00:09)
We nailed it, we have some good takes. That was so much fun. Happy holidays to you and your family. I’m happy to see you in person. Thanks for making the trip and being here.

Actor Smith: (00:18)
Absolutely.

Presenter: (00:19)
Do you have like family traditions? Do you have holiday plans? What do you have coming up?

Actor Smith: (00:24)
This year, we’re going to get all the Smith family. We’re also getting Camila’s family. So when we do that and we have that multicultural Christmas, Christmas lasts a long time at our place because they celebrate on the 24th. Midnight you exchange presents. All right. So you have the big sit down dinner, you get the china out for the first time all year.

In [17]:
prompt_data =f"""
Human:
Generate a summary of the transcription in the <transcription></transcription> XML tags below.
Then, classify the transcription according to the closest category within 'drama', 'comedy', 'talk show', 'news', or 'sports'.

<transcription>
{f}
</transcription>

Assistant:
"""

In [18]:
response, latency = bedrock_utils.call_bedrock(bedrock, prompt_data)
print(response, "\n\n", "Inference time:", latency)

 Here is a summary of the transcription:

The transcription is an interview between a presenter and actor Actor Smith. They discuss Actor Smith's new book and journal, his consideration of running for governor of Texas and decision not to at this time, and his relief concert for Texas after a devastating winter storm. Actor Smith also talks about his family's holiday traditions. The presenter congratulates Actor Smith on the success of his memoir Green Lights, which was on the bestseller list for 50 weeks. They show a clip from Actor Smith's new animated film Sing 2. 

Based on the content, this appears to be an interview on a talk show. So I would categorize this transcription as a talk show. 

 Inference time: 0:00:08.872937


------

## 5. Create splash pages describing upcoming events

**Use Case:** A media and entertainment company wants to create HTML pages quickly and easily, to promote their upcoming events.

**Task:** Code Generation

**Prompt Technique:** Zero-shot

In [21]:
prompt_data ="""
There is an upcoming music concert presented by the company Music Promotions.
The event is targeting young audience in the age range between 18 and 40.
The event will be done in the Royal Music Teather.
There will be seat assignment and tickets can be bought trought the Music Promotions website.
The event is a concert of the band called Super Rockers.
The event is on June 30th 2023 and doors will be open since 20:00.

Based the this information, generate the HTML code for an attractive splash page promoting this event.
"""

In [22]:
response, latency = bedrock_utils.call_bedrock(bedrock, prompt_data)
#print(response, "\n\n", "Inference time:", latency)
from IPython.display import display, HTML
display(HTML(response))

------